# 🧠 Interactive Neural Network Visualization

## Inspired by 3Blue1Brown's Neural Network Explanations

This notebook provides an **interactive visualization** of how neural networks work from scratch. You'll be able to:

1. **See the network architecture** - Watch neurons and connections in real-time
2. **Understand forward propagation** - See how data flows through layers
3. **Visualize weights** - Watch weights change as the network learns
4. **Provide your own inputs** - Test the network with custom values
5. **Train step-by-step** - Watch the network learn one step at a time

---

## 📦 Setup and Imports

In [ ]:
# Install required packages if needed
# !pip install numpy matplotlib ipywidgets

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, FancyBboxPatch
from matplotlib.collections import LineCollection
import matplotlib.colors as mcolors
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

# Enable interactive matplotlib in Jupyter
%matplotlib widget

print("✅ All packages loaded successfully!")

## 🔧 Neural Network from Scratch

Let's build a flexible neural network class that stores all intermediate values for visualization.

In [ ]:
class InteractiveNeuralNetwork:
    """
    A neural network implementation designed for interactive visualization.
    Stores all intermediate activations, gradients, and weight updates.
    """
    
    def __init__(self, layer_sizes, activation='sigmoid'):
        """
        Initialize the neural network.
        
        Parameters:
        -----------
        layer_sizes : list
            Number of neurons in each layer. 
            Example: [4, 8, 4, 1] = 4 inputs, 2 hidden layers (8, 4), 1 output
        activation : str
            Activation function: 'sigmoid', 'relu', or 'tanh'
        """
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes)
        self.activation_name = activation
        
        # Initialize weights using Xavier initialization
        self.weights = []
        self.biases = []
        
        for i in range(self.num_layers - 1):
            # Xavier initialization for better convergence
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2.0 / (layer_sizes[i] + layer_sizes[i+1]))
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
        
        # Storage for visualization
        self.activations = []      # Activation at each layer
        self.z_values = []         # Pre-activation values
        self.gradients = []        # Gradients for each layer
        self.weight_updates = []   # Weight updates from last step
        
        # Training history
        self.loss_history = []
        self.accuracy_history = []
        self.weight_history = []
        self.epoch = 0
    
    def sigmoid(self, z):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, a):
        """Derivative of sigmoid."""
        return a * (1 - a)
    
    def relu(self, z):
        """ReLU activation function."""
        return np.maximum(0, z)
    
    def relu_derivative(self, a):
        """Derivative of ReLU."""
        return (a > 0).astype(float)
    
    def tanh_activation(self, z):
        """Tanh activation function."""
        return np.tanh(z)
    
    def tanh_derivative(self, a):
        """Derivative of tanh."""
        return 1 - a**2
    
    def activate(self, z, output_layer=False):
        """Apply activation function."""
        if output_layer:
            return self.sigmoid(z)  # Always sigmoid for output
        
        if self.activation_name == 'sigmoid':
            return self.sigmoid(z)
        elif self.activation_name == 'relu':
            return self.relu(z)
        elif self.activation_name == 'tanh':
            return self.tanh_activation(z)
        return z
    
    def activate_derivative(self, a):
        """Get derivative of activation."""
        if self.activation_name == 'sigmoid':
            return self.sigmoid_derivative(a)
        elif self.activation_name == 'relu':
            return self.relu_derivative(a)
        elif self.activation_name == 'tanh':
            return self.tanh_derivative(a)
        return np.ones_like(a)
    
    def forward(self, X):
        """
        Forward propagation - compute activations for all layers.
        
        Parameters:
        -----------
        X : ndarray
            Input data (n_samples, n_features)
            
        Returns:
        --------
        ndarray : Output predictions
        """
        self.activations = [X]
        self.z_values = []
        
        current = X
        for i in range(self.num_layers - 1):
            z = np.dot(current, self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            
            is_output = (i == self.num_layers - 2)
            current = self.activate(z, output_layer=is_output)
            self.activations.append(current)
        
        return current
    
    def backward(self, X, y, learning_rate=0.01):
        """
        Backward propagation - compute gradients and update weights.
        
        Parameters:
        -----------
        X : ndarray
            Input data
        y : ndarray
            True labels
        learning_rate : float
            Learning rate for gradient descent
        """
        m = X.shape[0]
        self.gradients = []
        self.weight_updates = []
        
        # Output layer error
        output = self.activations[-1]
        delta = output - y
        
        # Backpropagate through layers
        deltas = [delta]
        for i in range(self.num_layers - 2, 0, -1):
            delta = np.dot(delta, self.weights[i].T) * self.activate_derivative(self.activations[i])
            deltas.insert(0, delta)
        
        self.gradients = deltas
        
        # Update weights and biases
        for i in range(self.num_layers - 1):
            weight_update = learning_rate * np.dot(self.activations[i].T, deltas[i]) / m
            bias_update = learning_rate * np.sum(deltas[i], axis=0, keepdims=True) / m
            
            self.weight_updates.append(weight_update)
            
            self.weights[i] -= weight_update
            self.biases[i] -= bias_update
    
    def compute_loss(self, y_true, y_pred):
        """Binary cross-entropy loss."""
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def compute_accuracy(self, X, y):
        """Compute classification accuracy."""
        predictions = (self.forward(X) >= 0.5).astype(int)
        return np.mean(predictions == y)
    
    def train_step(self, X, y, learning_rate=0.01):
        """
        Perform one training step.
        
        Returns:
        --------
        tuple : (loss, accuracy)
        """
        output = self.forward(X)
        self.backward(X, y, learning_rate)
        
        loss = self.compute_loss(y, output)
        accuracy = (np.mean((output >= 0.5).astype(int) == y))
        
        self.loss_history.append(loss)
        self.accuracy_history.append(accuracy)
        self.epoch += 1
        
        return loss, accuracy
    
    def reset(self):
        """Reset the network to initial random weights."""
        self.__init__(self.layer_sizes, self.activation_name)
    
    def predict(self, X):
        """Make predictions."""
        return self.forward(X)
    
    def get_network_info(self):
        """Get information about the network."""
        total_params = sum(w.size + b.size for w, b in zip(self.weights, self.biases))
        return {
            'architecture': self.layer_sizes,
            'activation': self.activation_name,
            'total_parameters': total_params,
            'epochs_trained': self.epoch
        }

print("✅ InteractiveNeuralNetwork class defined!")

## 🎨 Network Visualizer

This creates beautiful 3Blue1Brown-style visualizations of the neural network.

In [ ]:
class NetworkVisualizer:
    """
    3Blue1Brown-inspired neural network visualizer.
    Creates beautiful, informative visualizations of neural networks.
    """
    
    # Color scheme (3Blue1Brown inspired)
    COLORS = {
        'background': '#1a1a2e',
        'positive': '#4ecdc4',      # Teal for positive weights
        'negative': '#ff6b6b',      # Coral for negative weights
        'neutral': '#95a5a6',       # Gray for neutral
        'text': '#ecf0f1',          # Light text
        'highlight': '#f39c12',     # Orange highlight
        'activation_low': '#2c3e50',
        'activation_high': '#3498db',
        'input': '#9b59b6',         # Purple for input layer
        'output': '#2ecc71',        # Green for output layer
    }
    
    def __init__(self, network):
        """
        Initialize the visualizer.
        
        Parameters:
        -----------
        network : InteractiveNeuralNetwork
            The neural network to visualize
        """
        self.network = network
        self.neuron_positions = self._compute_positions()
    
    def _compute_positions(self):
        """Compute x, y positions for each neuron."""
        positions = []
        n_layers = len(self.network.layer_sizes)
        
        for layer_idx, n_neurons in enumerate(self.network.layer_sizes):
            layer_pos = []
            x = layer_idx / (n_layers - 1) if n_layers > 1 else 0.5
            
            for neuron_idx in range(n_neurons):
                if n_neurons == 1:
                    y = 0.5
                else:
                    y = 0.1 + 0.8 * (neuron_idx / (n_neurons - 1))
                layer_pos.append((x, y))
            positions.append(layer_pos)
        
        return positions
    
    def _weight_to_color(self, weight, max_weight):
        """Convert weight value to color."""
        if max_weight == 0:
            max_weight = 1
        
        normalized = np.clip(weight / max_weight, -1, 1)
        
        if normalized >= 0:
            alpha = 0.2 + 0.8 * normalized
            return (*mcolors.to_rgb(self.COLORS['positive']), alpha)
        else:
            alpha = 0.2 + 0.8 * abs(normalized)
            return (*mcolors.to_rgb(self.COLORS['negative']), alpha)
    
    def _activation_to_color(self, activation):
        """Convert activation value to color."""
        activation = np.clip(activation, 0, 1)
        
        # Interpolate between dark and bright blue
        low = np.array(mcolors.to_rgb(self.COLORS['activation_low']))
        high = np.array(mcolors.to_rgb(self.COLORS['activation_high']))
        
        color = low + activation * (high - low)
        return (*color, 0.9)
    
    def draw_network(self, ax, input_values=None, show_weights=True, 
                     show_values=True, show_gradients=False, title=None):
        """
        Draw the neural network.
        
        Parameters:
        -----------
        ax : matplotlib axes
            Axes to draw on
        input_values : ndarray, optional
            Input to propagate through network
        show_weights : bool
            Show weight connections
        show_values : bool
            Show activation values in neurons
        show_gradients : bool
            Show gradient values (if available)
        title : str, optional
            Title for the plot
        """
        ax.clear()
        ax.set_facecolor(self.COLORS['background'])
        ax.set_xlim(-0.1, 1.1)
        ax.set_ylim(-0.05, 1.05)
        ax.axis('off')
        
        # Forward pass if input provided
        if input_values is not None:
            if input_values.ndim == 1:
                input_values = input_values.reshape(1, -1)
            self.network.forward(input_values)
        
        # Get max weight for normalization
        max_weight = max(np.abs(w).max() for w in self.network.weights) if self.network.weights else 1
        
        # Draw connections (weights)
        if show_weights:
            for layer_idx in range(len(self.network.weights)):
                weights = self.network.weights[layer_idx]
                
                for i, pos1 in enumerate(self.neuron_positions[layer_idx]):
                    for j, pos2 in enumerate(self.neuron_positions[layer_idx + 1]):
                        weight = weights[i, j]
                        color = self._weight_to_color(weight, max_weight)
                        linewidth = 0.5 + 2.5 * abs(weight) / max_weight
                        
                        ax.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]],
                               color=color, linewidth=linewidth, zorder=1)
        
        # Draw neurons
        neuron_radius = 0.03
        
        for layer_idx, layer_pos in enumerate(self.neuron_positions):
            is_input = (layer_idx == 0)
            is_output = (layer_idx == len(self.network.layer_sizes) - 1)
            
            for neuron_idx, (x, y) in enumerate(layer_pos):
                # Get activation value
                if self.network.activations and layer_idx < len(self.network.activations):
                    act = self.network.activations[layer_idx]
                    value = float(act[0, neuron_idx]) if act.ndim > 1 else float(act[neuron_idx])
                    color = self._activation_to_color(value)
                else:
                    value = 0
                    color = self._activation_to_color(0)
                
                # Special colors for input/output layers
                if is_input:
                    edge_color = self.COLORS['input']
                elif is_output:
                    edge_color = self.COLORS['output']
                else:
                    edge_color = 'white'
                
                # Draw neuron
                circle = Circle((x, y), neuron_radius, facecolor=color,
                              edgecolor=edge_color, linewidth=2, zorder=5)
                ax.add_patch(circle)
                
                # Show value
                if show_values and self.network.activations:
                    ax.text(x, y, f'{value:.2f}', ha='center', va='center',
                           fontsize=7, color='white', fontweight='bold', zorder=6)
        
        # Layer labels
        layer_names = ['Input'] + [f'Hidden {i+1}' for i in range(len(self.network.layer_sizes) - 2)] + ['Output']
        for idx, name in enumerate(layer_names):
            x = idx / (len(self.network.layer_sizes) - 1) if len(self.network.layer_sizes) > 1 else 0.5
            ax.text(x, -0.02, name, ha='center', va='top', fontsize=10,
                   color=self.COLORS['text'], fontweight='bold')
            ax.text(x, 1.02, f'{self.network.layer_sizes[idx]} neurons',
                   ha='center', va='bottom', fontsize=8, color=self.COLORS['text'], alpha=0.7)
        
        # Title
        if title:
            ax.set_title(title, color=self.COLORS['text'], fontsize=14, fontweight='bold', pad=15)
    
    def draw_weights_heatmap(self, ax, layer_idx=0):
        """Draw weight matrix as heatmap."""
        ax.clear()
        ax.set_facecolor(self.COLORS['background'])
        
        if layer_idx >= len(self.network.weights):
            return
        
        weights = self.network.weights[layer_idx]
        max_val = np.abs(weights).max()
        
        im = ax.imshow(weights.T, cmap='RdBu_r', aspect='auto',
                      vmin=-max_val, vmax=max_val)
        
        ax.set_title(f'Weights: Layer {layer_idx} → {layer_idx + 1}',
                    color=self.COLORS['text'], fontsize=11)
        ax.set_xlabel(f'From Layer {layer_idx}', color=self.COLORS['text'])
        ax.set_ylabel(f'To Layer {layer_idx + 1}', color=self.COLORS['text'])
        ax.tick_params(colors=self.COLORS['text'])
        
        return im
    
    def draw_training_progress(self, ax_loss, ax_acc):
        """Draw training curves."""
        # Loss curve
        ax_loss.clear()
        ax_loss.set_facecolor(self.COLORS['background'])
        if self.network.loss_history:
            ax_loss.plot(self.network.loss_history, color=self.COLORS['negative'], linewidth=2)
        ax_loss.set_title('Loss', color=self.COLORS['text'], fontsize=10)
        ax_loss.set_xlabel('Epoch', color=self.COLORS['text'], fontsize=8)
        ax_loss.tick_params(colors=self.COLORS['text'])
        ax_loss.grid(True, alpha=0.2)
        
        # Accuracy curve
        ax_acc.clear()
        ax_acc.set_facecolor(self.COLORS['background'])
        if self.network.accuracy_history:
            ax_acc.plot(self.network.accuracy_history, color=self.COLORS['output'], linewidth=2)
        ax_acc.set_title('Accuracy', color=self.COLORS['text'], fontsize=10)
        ax_acc.set_xlabel('Epoch', color=self.COLORS['text'], fontsize=8)
        ax_acc.tick_params(colors=self.COLORS['text'])
        ax_acc.grid(True, alpha=0.2)
        ax_acc.set_ylim(0, 1)

print("✅ NetworkVisualizer class defined!")

## 📊 Create Sample Data

Let's create some data for our neural network to learn.

In [ ]:
def create_sample_data(n_samples=200, n_features=4, random_state=42):
    """
    Create sample binary classification data.
    """
    np.random.seed(random_state)
    
    n_per_class = n_samples // 2
    
    # Class 0: lower values
    X0 = np.random.randn(n_per_class, n_features) * 0.5 - 0.5
    
    # Class 1: higher values
    X1 = np.random.randn(n_per_class, n_features) * 0.5 + 0.5
    
    X = np.vstack([X0, X1])
    y = np.array([0] * n_per_class + [1] * n_per_class).reshape(-1, 1)
    
    # Shuffle
    indices = np.random.permutation(n_samples)
    X = X[indices]
    y = y[indices]
    
    # Normalize to 0-1
    X = (X - X.min()) / (X.max() - X.min())
    
    return X, y

# Create data
X_train, y_train = create_sample_data(n_samples=200, n_features=4)

print(f"✅ Created training data:")
print(f"   - Samples: {X_train.shape[0]}")
print(f"   - Features: {X_train.shape[1]}")
print(f"   - Classes: {len(np.unique(y_train))}")

## 🚀 Interactive Visualization Dashboard

This is the main interactive interface! You can:
- **Train** the network step-by-step or in batches
- **Reset** to start fresh
- **Input custom values** and see the prediction
- **Watch weights change** as training progresses

In [ ]:
# Create the neural network
nn = InteractiveNeuralNetwork(layer_sizes=[4, 6, 4, 1], activation='sigmoid')
viz = NetworkVisualizer(nn)

print("✅ Neural Network created!")
print(f"   Architecture: {nn.layer_sizes}")
print(f"   Total parameters: {nn.get_network_info()['total_parameters']}")

In [ ]:
# Create interactive dashboard using ipywidgets

# Output areas
output_network = widgets.Output()
output_info = widgets.Output()

# Create the figure
with output_network:
    fig = plt.figure(figsize=(14, 10), facecolor='#1a1a2e')
    
    # Main network visualization
    ax_network = fig.add_axes([0.05, 0.35, 0.55, 0.58])
    
    # Training curves
    ax_loss = fig.add_axes([0.65, 0.55, 0.3, 0.18])
    ax_acc = fig.add_axes([0.65, 0.78, 0.3, 0.15])
    
    # Weight matrix
    ax_weights = fig.add_axes([0.65, 0.35, 0.3, 0.15])
    
    plt.show()

def update_visualization(sample_idx=0, layer_idx=0):
    """Update the visualization."""
    with output_network:
        # Get sample
        input_val = X_train[sample_idx:sample_idx+1]
        
        # Draw network
        viz.draw_network(ax_network, input_values=input_val, show_weights=True, show_values=True,
                        title=f'Neural Network | Epoch: {nn.epoch} | Sample: {sample_idx}')
        
        # Draw training progress
        viz.draw_training_progress(ax_loss, ax_acc)
        
        # Draw weight matrix
        viz.draw_weights_heatmap(ax_weights, layer_idx)
        
        fig.canvas.draw_idle()

# Create widgets
learning_rate_slider = widgets.FloatSlider(
    value=0.1, min=0.001, max=1.0, step=0.01,
    description='Learning Rate:', style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

sample_slider = widgets.IntSlider(
    value=0, min=0, max=len(X_train)-1, step=1,
    description='Sample Index:', style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

layer_slider = widgets.IntSlider(
    value=0, min=0, max=len(nn.weights)-1, step=1,
    description='Weight Layer:', style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

epochs_input = widgets.IntText(
    value=100, description='Epochs:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px')
)

custom_input = widgets.Text(
    value='0.5, 0.5, 0.5, 0.5',
    description='Custom Input:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Buttons
train_button = widgets.Button(description='Train', button_style='success', layout=widgets.Layout(width='100px'))
step_button = widgets.Button(description='Step', button_style='info', layout=widgets.Layout(width='100px'))
reset_button = widgets.Button(description='Reset', button_style='danger', layout=widgets.Layout(width='100px'))
forward_button = widgets.Button(description='Forward Pass', button_style='primary', layout=widgets.Layout(width='120px'))

# Status label
status_label = widgets.HTML(value='<b>Ready to train!</b>')

def on_train_click(b):
    """Train for multiple epochs."""
    lr = learning_rate_slider.value
    epochs = epochs_input.value
    
    status_label.value = f'<b>Training for {epochs} epochs...</b>'
    
    for _ in range(epochs):
        loss, acc = nn.train_step(X_train, y_train, lr)
    
    status_label.value = f'<b>Epoch {nn.epoch} | Loss: {loss:.4f} | Accuracy: {acc:.4f}</b>'
    update_visualization(sample_slider.value, layer_slider.value)

def on_step_click(b):
    """Train for one step."""
    lr = learning_rate_slider.value
    loss, acc = nn.train_step(X_train, y_train, lr)
    
    status_label.value = f'<b>Epoch {nn.epoch} | Loss: {loss:.4f} | Accuracy: {acc:.4f}</b>'
    update_visualization(sample_slider.value, layer_slider.value)

def on_reset_click(b):
    """Reset the network."""
    nn.reset()
    status_label.value = '<b>Network reset! Ready to train.</b>'
    update_visualization(sample_slider.value, layer_slider.value)

def on_forward_click(b):
    """Forward pass with custom input."""
    try:
        values = [float(x.strip()) for x in custom_input.value.split(',')]
        if len(values) == nn.layer_sizes[0]:
            input_arr = np.array([values])
            output = nn.forward(input_arr)
            
            with output_network:
                viz.draw_network(ax_network, input_values=input_arr, show_weights=True, show_values=True,
                               title=f'Custom Input | Output: {output[0,0]:.4f}')
                fig.canvas.draw_idle()
            
            prediction = 'Class 1' if output[0,0] >= 0.5 else 'Class 0'
            status_label.value = f'<b>Output: {output[0,0]:.4f} → {prediction}</b>'
        else:
            status_label.value = f'<b style="color:red">Error: Need {nn.layer_sizes[0]} input values</b>'
    except Exception as e:
        status_label.value = f'<b style="color:red">Error: {str(e)}</b>'

def on_slider_change(change):
    """Handle slider changes."""
    update_visualization(sample_slider.value, layer_slider.value)

# Connect callbacks
train_button.on_click(on_train_click)
step_button.on_click(on_step_click)
reset_button.on_click(on_reset_click)
forward_button.on_click(on_forward_click)
sample_slider.observe(on_slider_change, names='value')
layer_slider.observe(on_slider_change, names='value')

# Layout
controls_row1 = widgets.HBox([learning_rate_slider, epochs_input])
controls_row2 = widgets.HBox([sample_slider, layer_slider])
buttons_row = widgets.HBox([train_button, step_button, reset_button])
custom_row = widgets.HBox([custom_input, forward_button])

controls = widgets.VBox([
    widgets.HTML('<h3>🎛️ Controls</h3>'),
    controls_row1,
    controls_row2,
    buttons_row,
    widgets.HTML('<h4>🔬 Test Custom Input</h4>'),
    custom_row,
    widgets.HTML('<hr>'),
    status_label
])

# Initial visualization
update_visualization(0, 0)

# Display
display(controls)
display(output_network)

## 🎬 Step-by-Step Forward Propagation Visualization

Watch how data flows through each layer of the network!

In [ ]:
def visualize_forward_step_by_step(network, input_values, delay=1.0):
    """
    Visualize forward propagation step by step.
    
    Parameters:
    -----------
    network : InteractiveNeuralNetwork
        The neural network
    input_values : ndarray
        Input values (should be 1D or 2D with shape (1, n_features))
    delay : float
        Delay between steps in seconds
    """
    import time
    from IPython.display import clear_output
    
    if input_values.ndim == 1:
        input_values = input_values.reshape(1, -1)
    
    viz_local = NetworkVisualizer(network)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6), facecolor='#1a1a2e')
    ax_net = axes[0]
    ax_info = axes[1]
    
    print("\n" + "="*60)
    print("  FORWARD PROPAGATION VISUALIZATION")
    print("="*60)
    print(f"\nInput: {input_values[0]}\n")
    
    # Step through each layer
    current = input_values
    network.activations = [current]
    
    for layer_idx in range(network.num_layers - 1):
        # Compute this layer
        z = np.dot(current, network.weights[layer_idx]) + network.biases[layer_idx]
        
        is_output = (layer_idx == network.num_layers - 2)
        if is_output:
            current = network.sigmoid(z)
        else:
            current = network.activate(z)
        
        network.activations.append(current)
        
        # Clear and redraw
        ax_net.clear()
        ax_info.clear()
        
        # Draw network
        viz_local.draw_network(ax_net, show_weights=True, show_values=True,
                              title=f'Layer {layer_idx} → Layer {layer_idx + 1}')
        
        # Info panel
        ax_info.set_facecolor('#1a1a2e')
        ax_info.axis('off')
        
        info_text = f"""
        LAYER {layer_idx + 1} COMPUTATION
        {'='*30}
        
        Input shape: {network.activations[layer_idx].shape}
        Weight shape: {network.weights[layer_idx].shape}
        
        z = input × weights + bias
        a = activation(z)
        
        Output: {current[0]}
        """
        
        ax_info.text(0.1, 0.5, info_text, transform=ax_info.transAxes,
                    fontsize=11, color='white', family='monospace',
                    verticalalignment='center')
        
        fig.canvas.draw()
        plt.pause(delay)
        
        print(f"Layer {layer_idx + 1}: {current[0]}")
    
    # Final result
    output = network.activations[-1][0, 0]
    prediction = "Class 1" if output >= 0.5 else "Class 0"
    
    print(f"\n{'='*60}")
    print(f"FINAL OUTPUT: {output:.4f}")
    print(f"PREDICTION: {prediction}")
    print(f"{'='*60}\n")
    
    plt.show()
    return output

print("✅ Forward propagation visualizer ready!")
print("Run the next cell to see step-by-step propagation.")

In [ ]:
# Example: Visualize forward propagation step by step
test_input = np.array([0.2, 0.8, 0.5, 0.3])
print(f"Testing with input: {test_input}")

output = visualize_forward_step_by_step(nn, test_input, delay=1.5)

## 📈 Weight Evolution Visualization

Watch how weights evolve during training!

In [ ]:
def visualize_training_evolution(layer_sizes=[4, 6, 4, 1], n_epochs=500, lr=0.1, 
                                  snapshot_every=50):
    """
    Visualize how the network evolves during training.
    """
    # Create fresh network
    net = InteractiveNeuralNetwork(layer_sizes, activation='sigmoid')
    
    # Store weight snapshots
    weight_snapshots = []
    epochs_recorded = []
    
    print(f"Training for {n_epochs} epochs...")
    
    for epoch in range(n_epochs):
        loss, acc = net.train_step(X_train, y_train, lr)
        
        if epoch % snapshot_every == 0:
            weight_snapshots.append([w.copy() for w in net.weights])
            epochs_recorded.append(epoch)
            print(f"  Epoch {epoch}: Loss={loss:.4f}, Accuracy={acc:.4f}")
    
    # Add final snapshot
    weight_snapshots.append([w.copy() for w in net.weights])
    epochs_recorded.append(n_epochs)
    
    # Visualize snapshots
    n_snapshots = len(weight_snapshots)
    n_layers = len(layer_sizes) - 1
    
    fig, axes = plt.subplots(n_layers, min(n_snapshots, 6), figsize=(16, 4*n_layers),
                            facecolor='#1a1a2e')
    
    if n_layers == 1:
        axes = axes.reshape(1, -1)
    
    # Select subset of snapshots to show
    indices = np.linspace(0, n_snapshots-1, min(n_snapshots, 6), dtype=int)
    
    for row, layer_idx in enumerate(range(n_layers)):
        for col, snap_idx in enumerate(indices):
            ax = axes[row, col] if n_layers > 1 else axes[col]
            ax.set_facecolor('#1a1a2e')
            
            weights = weight_snapshots[snap_idx][layer_idx]
            max_val = max(np.abs(w).max() for snapshot in weight_snapshots for w in snapshot)
            
            im = ax.imshow(weights.T, cmap='RdBu_r', aspect='auto',
                          vmin=-max_val, vmax=max_val)
            
            ax.set_title(f'Epoch {epochs_recorded[snap_idx]}', color='white', fontsize=10)
            
            if col == 0:
                ax.set_ylabel(f'Layer {layer_idx}→{layer_idx+1}', color='white')
            
            ax.tick_params(colors='white')
    
    plt.suptitle('Weight Evolution During Training', color='white', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Also show training curves
    fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), facecolor='#1a1a2e')
    
    ax1.set_facecolor('#1a1a2e')
    ax1.plot(net.loss_history, color='#ff6b6b', linewidth=2)
    ax1.set_title('Loss During Training', color='white')
    ax1.set_xlabel('Epoch', color='white')
    ax1.set_ylabel('Loss', color='white')
    ax1.tick_params(colors='white')
    ax1.grid(True, alpha=0.2)
    
    ax2.set_facecolor('#1a1a2e')
    ax2.plot(net.accuracy_history, color='#2ecc71', linewidth=2)
    ax2.set_title('Accuracy During Training', color='white')
    ax2.set_xlabel('Epoch', color='white')
    ax2.set_ylabel('Accuracy', color='white')
    ax2.tick_params(colors='white')
    ax2.grid(True, alpha=0.2)
    ax2.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
    return net

print("✅ Weight evolution visualizer ready!")
print("Run the next cell to train and visualize weight evolution.")

In [ ]:
# Train and visualize weight evolution
trained_net = visualize_training_evolution(
    layer_sizes=[4, 6, 4, 1],
    n_epochs=500,
    lr=0.2,
    snapshot_every=100
)

## 🎯 Interactive Custom Network Builder

Build your own network architecture and watch it learn!

In [ ]:
# Interactive network builder
@interact(
    hidden1=widgets.IntSlider(min=2, max=10, value=6, description='Hidden 1:'),
    hidden2=widgets.IntSlider(min=2, max=10, value=4, description='Hidden 2:'),
    activation=widgets.Dropdown(options=['sigmoid', 'relu', 'tanh'], value='sigmoid', description='Activation:'),
    learning_rate=widgets.FloatSlider(min=0.01, max=1.0, value=0.1, step=0.01, description='LR:'),
    epochs=widgets.IntSlider(min=100, max=2000, value=500, step=100, description='Epochs:')
)
def build_and_train(hidden1, hidden2, activation, learning_rate, epochs):
    """Build and train a custom network."""
    
    # Create network with custom architecture
    layer_sizes = [4, hidden1, hidden2, 1]
    net = InteractiveNeuralNetwork(layer_sizes, activation=activation)
    
    print(f"\nNetwork Architecture: {layer_sizes}")
    print(f"Activation: {activation}")
    print(f"Training for {epochs} epochs with LR={learning_rate}...\n")
    
    # Train
    for _ in range(epochs):
        loss, acc = net.train_step(X_train, y_train, learning_rate)
    
    print(f"Final Loss: {loss:.4f}")
    print(f"Final Accuracy: {acc:.4f}")
    
    # Visualize
    viz = NetworkVisualizer(net)
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5), facecolor='#1a1a2e')
    
    # Network structure
    viz.draw_network(axes[0], input_values=X_train[0:1], show_weights=True, show_values=True,
                    title='Network Architecture')
    
    # Loss curve
    axes[1].set_facecolor('#1a1a2e')
    axes[1].plot(net.loss_history, color='#ff6b6b', linewidth=2)
    axes[1].set_title('Loss', color='white')
    axes[1].set_xlabel('Epoch', color='white')
    axes[1].tick_params(colors='white')
    axes[1].grid(True, alpha=0.2)
    
    # Accuracy curve
    axes[2].set_facecolor('#1a1a2e')
    axes[2].plot(net.accuracy_history, color='#2ecc71', linewidth=2)
    axes[2].set_title('Accuracy', color='white')
    axes[2].set_xlabel('Epoch', color='white')
    axes[2].tick_params(colors='white')
    axes[2].grid(True, alpha=0.2)
    axes[2].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

## 🔍 Understanding Neurons: Activation Exploration

See how different neurons respond to different inputs!

In [ ]:
def explore_neuron_activations(network, n_samples=100):
    """
    Explore how neurons activate across different inputs.
    """
    # Generate random inputs
    np.random.seed(42)
    inputs = np.random.rand(n_samples, network.layer_sizes[0])
    
    # Collect activations
    all_activations = []
    for i in range(n_samples):
        network.forward(inputs[i:i+1])
        all_activations.append([a.copy() for a in network.activations])
    
    # Visualize activation distributions for each layer
    n_layers = len(network.layer_sizes)
    fig, axes = plt.subplots(1, n_layers, figsize=(4*n_layers, 4), facecolor='#1a1a2e')
    
    for layer_idx in range(n_layers):
        ax = axes[layer_idx]
        ax.set_facecolor('#1a1a2e')
        
        # Get activations for this layer across all samples
        layer_acts = np.array([act[layer_idx][0] for act in all_activations])
        
        # Box plot for each neuron
        bp = ax.boxplot(layer_acts, patch_artist=True)
        
        colors = plt.cm.viridis(np.linspace(0, 1, network.layer_sizes[layer_idx]))
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
        
        ax.set_title(f'Layer {layer_idx}', color='white')
        ax.set_xlabel('Neuron', color='white')
        ax.set_ylabel('Activation', color='white')
        ax.tick_params(colors='white')
        ax.grid(True, alpha=0.2)
    
    plt.suptitle('Neuron Activation Distributions Across Inputs', color='white', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

# First train the network if needed
if nn.epoch < 100:
    print("Training network first...")
    for _ in range(200):
        nn.train_step(X_train, y_train, 0.1)

print(f"\nExploring activations after {nn.epoch} training epochs:")
explore_neuron_activations(nn, n_samples=100)

## 🎓 Educational Summary

### What You've Learned:

1. **Network Architecture**: How neurons are organized in layers
2. **Forward Propagation**: How data flows through the network
3. **Weights & Biases**: How they determine the network's behavior
4. **Activation Functions**: How neurons "fire" based on their inputs
5. **Training**: How the network learns by adjusting weights
6. **Loss & Accuracy**: How we measure the network's performance

### Key Concepts Visualized:

- **Blue connections** = Positive weights (enhance the signal)
- **Red connections** = Negative weights (inhibit the signal)
- **Thick lines** = Strong weights (important connections)
- **Bright neurons** = High activation (neuron is "firing")
- **Dark neurons** = Low activation (neuron is quiet)

---

Try experimenting with different:
- Network architectures (number of layers, neurons per layer)
- Learning rates (faster vs. slower learning)
- Activation functions (sigmoid, relu, tanh)
- Custom inputs to see how predictions change!

In [ ]:
# Final summary
print("\n" + "="*60)
print("  INTERACTIVE NEURAL NETWORK VISUALIZATION")
print("  Inspired by 3Blue1Brown")
print("="*60)
print("\nFeatures:")
print("  ✓ Real-time network visualization")
print("  ✓ Interactive training controls")
print("  ✓ Step-by-step forward propagation")
print("  ✓ Weight evolution visualization")
print("  ✓ Custom input testing")
print("  ✓ Flexible network architecture")
print("\nTry modifying the code to:")
print("  - Add more hidden layers")
print("  - Change activation functions")
print("  - Test with different datasets")
print("  - Create your own visualizations!")
print("\n" + "="*60)